In [1]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:


import pandas as pd
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

import seaborn as sns
import matplotlib.pyplot as plt
import datatable as dt


In [3]:
SEED = 123

data = dt.fread('~/jane-street-market-prediction/train.csv').to_pandas()


In [4]:
profile = pd.pivot_table(data, index='date', values='resp',
           aggfunc={'resp':[np.sum, np.std, np.mean]},fill_value=0)
profile.sort_values(by='std')
vol_bins = pd.qcut(profile['std'], 10, labels=False)

In [5]:
data['vol_bin'] = 0
for vb in np.unique(vol_bins):
    mask = vol_bins == vb
    date_mask = data['date'].isin(vol_bins.loc[mask].index)
    data.loc[date_mask,'vol_bin'] = vb

In [6]:
data.fillna(data.mean(),inplace=True)

data['action'] = ((data['resp'].values) < 0).astype(int)


features = [c for c in data.columns if "feature" in c]
features.append('weight')

In [9]:
len(features)

131

In [10]:
f_mean = np.mean(data[features[1:]].values,axis=0)


target_feature = 'vol_bin'
X_train = data.loc[:, features]
y_train = (data.loc[:, target_feature])

In [11]:
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    #x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x) #(x)
    
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
        x = tf.keras.layers.Concatenate(axis=1)([x, inp])
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

In [12]:


batch_size = 5000
hidden_units = [130, 130, 130]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

len(np.unique(y_train))


clf = create_mlp(
    len(features),len(np.unique(y_train)), hidden_units, dropout_rates, label_smoothing, learning_rate
    )

clf.fit(X_train, y_train, epochs=200, batch_size=20000)

opt_th = 0.5000


Epoch 1/200
120/120 [==============================] - 5s 21ms/step - loss: -115.6602 - AUC: 0.6262
Epoch 2/200
120/120 [==============================] - 3s 21ms/step - loss: -3834.5740 - AUC: 0.6329
Epoch 3/200
120/120 [==============================] - 3s 21ms/step - loss: -22864.2302 - AUC: 0.6149
Epoch 4/200
120/120 [==============================] - 2s 21ms/step - loss: -70245.9058 - AUC: 0.5799
Epoch 5/200
120/120 [==============================] - 2s 21ms/step - loss: -157560.9855 - AUC: 0.5525
Epoch 6/200
120/120 [==============================] - 2s 21ms/step - loss: -295241.2057 - AUC: 0.5333
Epoch 7/200
120/120 [==============================] - 3s 21ms/step - loss: -492645.6937 - AUC: 0.5225
Epoch 8/200
120/120 [==============================] - 3s 22ms/step - loss: -756204.7712 - AUC: 0.5161
Epoch 9/200
120/120 [==============================] - 3s 21ms/step - loss: -1094205.5883 - AUC: 0.5117
Epoch 10/200
120/120 [==============================] - 3s 21ms/step - loss: -1

120/120 [==============================] - 3s 21ms/step - loss: -347742985.7851 - AUC: 0.5010
Epoch 79/200
120/120 [==============================] - 3s 21ms/step - loss: -358829717.1570 - AUC: 0.5011
Epoch 80/200
120/120 [==============================] - 3s 21ms/step - loss: -370451730.2479 - AUC: 0.5011
Epoch 81/200
120/120 [==============================] - 3s 21ms/step - loss: -381786844.0331 - AUC: 0.5012
Epoch 82/200
120/120 [==============================] - 3s 21ms/step - loss: -393379334.0826 - AUC: 0.5012
Epoch 83/200
120/120 [==============================] - 3s 21ms/step - loss: -405775325.0909 - AUC: 0.5011
Epoch 84/200
120/120 [==============================] - 3s 21ms/step - loss: -417940763.2397 - AUC: 0.5011
Epoch 85/200
120/120 [==============================] - 3s 21ms/step - loss: -430326722.6446 - AUC: 0.5012
Epoch 86/200
120/120 [==============================] - 3s 22ms/step - loss: -443078502.8760 - AUC: 0.5011
Epoch 87/200
120/120 [============================

120/120 [==============================] - 2s 21ms/step - loss: -1899420484.7603 - AUC: 0.5011
Epoch 154/200
120/120 [==============================] - 2s 21ms/step - loss: -1929927298.1157 - AUC: 0.5011
Epoch 155/200
120/120 [==============================] - 2s 21ms/step - loss: -1963542895.0744 - AUC: 0.5012
Epoch 156/200
120/120 [==============================] - 2s 21ms/step - loss: -1996521753.3884 - AUC: 0.5011
Epoch 157/200
120/120 [==============================] - 2s 21ms/step - loss: -2030158545.4545 - AUC: 0.5011
Epoch 158/200
120/120 [==============================] - 2s 21ms/step - loss: -2062246091.1074 - AUC: 0.5010
Epoch 159/200
120/120 [==============================] - 3s 21ms/step - loss: -2095941153.8512 - AUC: 0.5011
Epoch 160/200
120/120 [==============================] - 3s 21ms/step - loss: -2131578322.5124 - AUC: 0.5011
Epoch 161/200
120/120 [==============================] - 3s 21ms/step - loss: -2164261435.2397 - AUC: 0.5011
Epoch 162/200
120/120 [==========

In [ ]:
from math import sqrt

def utility_score(df,action_vec):
    """Calculate utility score of a dataframe according to formulas defined at
    https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation
    """

    p = df['weight']  * df['resp'] * action_vec
    df['p'] = p
    p_i = df.set_index('date')['p'].groupby('date').sum()
    t = (p_i.sum() / sqrt((p_i**2).sum())) * (sqrt(250 / p_i.index.size))
    return min(max(t, 0), 6) * p_i.sum()

In [ ]:
pred_df = train[['date', 'resp','weight']]
pred_df['actual'] = ((train['resp'].values) > 0).astype(int)
preds = clf(X_train.values, training=False)
pred_df['preds'] = preds.numpy()
pred_df['action'] = (pred_df['preds'] >= 0.5) * 1

In [ ]:
utility_score(pred_df, pred_df['action']) / utility_score(pred_df, pred_df['actual'])
